# Simple RAG example with Langchain, Ollama and and open-source model

Sources:

* https://github.com/svpino/llm
* https://github.com/AIAnytime/Gemma-7B-RAG-using-Ollama/blob/main/Ollama%20Gemma.ipynb
* https://www.youtube.com/watch?v=-MexTC18h20&ab_channel=AIAnytime
* https://www.youtube.com/watch?v=HRvyei7vFSM&ab_channel=Underfitted

 
# Requirements

* Ollama installed locally

In [4]:
!pip3 install langchain
#langchain-openai
!pip3 install langchain_pinecone
!pip3 install pip install langchain[docarray]
!pip3 install docarray
#pydantic==1.10.8
#pytube 
#python-dotenv
#tiktoken 
#pinecone-client 
!pip3 install scikit-learn
#ruff
!pip3 install pypdf
#git+https://github.com/openai/whisper.git

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 215 kB 5.2 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 10.9 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
zsh:1: no matches found: langchain[docarray]
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 270 kB 5.4 MB/s eta 0:00:01
     |████████████████████████████████| 240 kB 34.1 MB/s eta 0:00:01
     |████████████████████████████████| 87 kB 8.8 MB/s  eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip'

In [22]:
#import os
#from dotenv import load_dotenv

#load_dotenv()

#OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
#MODEL = "gpt-3.5-turbo"
#MODEL = "mixtral:8x7b"
MODEL = "gemma:7b"
MODEL = "llama2"

In [23]:
from langchain_community.llms import Ollama
#from langchain_openai.chat_models import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
#from langchain_openai.embeddings import OpenAIEmbeddings

#if MODEL.startswith("gpt"):
#    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
#    embeddings = OpenAIEmbeddings()
#else:
model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

model.invoke("Tell me a joke")

"\nSure! Here's a classic one:\n\nWhy don't scientists trust atoms?\nBecause they make up everything!\n\nI hope you found that amusing! Do you want to hear another one?"

In [8]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
chain.invoke("Tell me a joke")

"Sure, here's one:\n\nWhy don't scientists trust atoms?\nBecause they make up everything!\n\nI hope that brought a smile to your face!"

In [10]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [11]:
chain = prompt | model | parser

chain.invoke({"context": "My parents named me Santiago", "question": "What's your name'?"})

' Sure! Based on the context you provided, my answer would be:\n\n"My name is Santiago."'

In [12]:
# Example pdf downloaded from https://www.ml.school/ 
from langchain_community.document_loaders import PyPDFLoader


loader = PyPDFLoader("machine_learning.pdf")
pages = loader.load_and_split()
pages

[Document(page_content='11/4/24, 19:02 Building Machine Learning Systems That Don\'t Suck\nhttps://www.ml.school 1/10Building Machine Learning Systems That Don\'t\nSuck\nA live, interactive program that\'ll help you build production-readymachine\nlearning systems from the ground up.\nNext cohort:\xa0May6 - 23, 2024\nCheck the schedulefor more details about upcoming cohorts.\nI want to join!Sign in\nLearn how to design, build, deploy, and scale machine learning\nsystems to solve real-world problems.\nI\'ll lose my mind if I see another book or course teaching people the same basic\nideas for the hundredth time. Most people are stuck in beginner mode, and finding\nhelp to solve real-world problems is hard.\nI want to change that.\nI started writing software 30 years ago. I\'ve written pipelines and trained models\nfor some of the largest companies in the world. I want to show you how to do the\nsame.\nThis is the class I wish I had taken when I started."This is the best machine learning 

In [15]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

/Users/sergiopaniegoblanco/Library/Python/3.9/lib/python/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [16]:
retriever = vectorstore.as_retriever()
retriever.invoke("machine learning")

[Document(page_content="11/4/24, 19:02 Building Machine Learning Systems That Don't Suck\nhttps://www.ml.school 10/10I'm a machine learning engineer with over two decades of experience\nbuilding and scaling enterprise software and machine learning systems.\nI love neural networks. I love to make them work at scale.\nFrom 2009 to 2023, I built products for Disney, Boston Dynamics, IBM,\nDell, G4S, Anheuser-Busch, and NextEra Energy, among other clients. I\nlearned about trade-offs and how to create products that work.\nI started this program in March 2023. Since then, thousands of students\nhave graduated, and I can't wait to meet you in class.\nCopyright © 2024 Tideily LLC\nAll rights reserved.", metadata={'source': 'machine_learning.pdf', 'page': 9}),
 Document(page_content='11/4/24, 19:02 Building Machine Learning Systems That Don\'t Suck\nhttps://www.ml.school 8/10Hyperparameter tuning and experiment tracking.\nMeasuring the quality of your holdout set.\nAn introduction to distribut

In [17]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [18]:
questions = [
    "What is the purpose of the course?",
    "How many hours of live sessions?",
    "How many coding assignments are there in the program?",
    "Is there a program certificate upon completion?",
    "What programming language will be used in the program?",
    "How much does the program cost?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: What is the purpose of the course?
Answer:  Based on the context provided, the purpose of the course appears to be to teach participants how to build machine learning systems that work in real-world scenarios. The course covers a wide range of topics, including best practices for tackling common challenges faced by machine learning engineers, how to use various techniques such as active learning and distributed training, and how to create pipelines using Amazon SageMaker and open-source tools. The course also includes live, interactive sessions where participants can ask questions and interact with the instructor and other students in real time. Additionally, the course provides lifetime access to every past and future cohort, allowing participants to learn at their own pace and benefit from updates to the program.

Question: How many hours of live sessions?
Answer: Based on the context provided, the answer to the question "How many hours of live sessions?" is:

* Every itera